In [588]:
import numpy as np 
import pandas as pd 
import re
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import pymorphy2
import nltk
from nltk.corpus import stopwords
morph = pymorphy2.MorphAnalyzer()
ru_stopwords = stopwords.words('russian')

In [624]:
def df_preprocess(text):
    reg = re.compile('[^а-яА-яa-zA-Z0-9 ]')
    text = text.lower().replace("ё", "е")
    text = text.replace("ъ", "ь")
    text = text.replace("й", "и")
    text = reg.sub(' ', text)
    
    return morph.normal_forms(text)[0]

In [613]:
df = pd.read_csv('test_data.csv')

In [614]:
df.head(3)

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день


In [615]:
df['text'].sample(5)

238                                 Так
58                    Пробить через них
119    8 пользователей указан правильно
91                                  Ага
110                   Алло здравствуйте
Name: text, dtype: object

In [616]:
all_words = df['text'].str.split(expand=True).unstack().value_counts()
data = [go.Bar(
            x = all_words.index.values[2:50],
            y = all_words.values[2:50],
            marker= dict(colorscale='Jet',
                         color = all_words.values[2:100]
                        ),
            text='Word counts'
    )]

layout = go.Layout(
    title='Top 50 (Uncleaned) Word frequencies in the training dataset'
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='basic-bar')

In [625]:
client = {}
manager = {}
for id in df['dlg_id'].unique():
    client[id] = df[(df.dlg_id == id) & (df.role == 'client')]['text'].values
    manager[id] = df[(df.dlg_id == id) & (df.role == 'manager')]['text'].values
# manager[0]

In [626]:
manager_join = [None] * 6
for man in manager:
    manager_join[man] = '. '.join(manager[man])
manager_join[0]

'Алло здравствуйте. Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается. Угу ну возможно вы рассмотрите и другие варианты видите это хорошая практика сравнивать. Угу а на что вы обращаете внимание при выборе. Что для вас приоритет. Ну у вас срок заканчивается поэтому мы набрали + ко всему. А так нет не только поэтому просто я обратила внимание и вы пользуетесь виджетами мы там с вами сотрудничали. А если вы 19 являетесь то лучше то идти бесплатно. Ага хорошо. Индивидуальным поэтому не все то есть сотрудничают с кем то сейчас да кому то например не с кем сейчас справляться кто то сравнивал. Да мы услышали вас спасибо за рекомендации. Вот понимаю да но дело в том что в основном да за месяц. Необходимо обзванивать. Хорошо я данную информацию тогда зафиксирую передам будем иметь ввиду. И вам спасибо большое за обратную связь. Всего хорошего до свидания'

In [627]:
delete_words = df['text'].str.split(expand=True).unstack().value_counts().index.tolist()[:100]

In [628]:
text_list = []
dict_text = {}
for man in manager:
    for i in manager[man]:
        tk = nltk.word_tokenize(i)
        words = list(set(tk) - set(delete_words))
        text_list.append((df_preprocess(i)))
    dict_text[man] = '. '.join(text_list)
    text_list = []

In [629]:
texts = [None] * 6
for man in dict_text:
    texts[man] = dict_text[man]

In [667]:
def find_hi(text):
    return re.findall(r"([^.]*здравствуит[^.]*)", text) + re.findall(r"([^.]*добрыи[^.]*)", text)

def find_company(text):
    return (re.findall(r"([^.]*компания[^.]*)", text))

def find_goodbye(text):
    return (re.findall(r"([^.]*свидание[^.]*)", text)) + (re.findall(r"([^.]*добрый[^.]*)", text)) + re.findall(r"([^.]*свидания[^.]*)", text)

def find_name(text):
    return((re.findall(r"([^.]*зовут[^.]*)", text)) + (re.findall(r"([^.]*да это[^.]*)", text)))

In [669]:
print("Phrases with manager's:\n 1.greetings \n 2.name \n 3.company\n 4.goodbye\n\n")
n = 1
for manager in texts:
    print("From dialog", n, ":")
    print("1.", find_hi(manager))
    print("2.", find_name(manager))
    print("3.", find_company(manager))
    print("4.", find_goodbye(manager))
    if (find_hi(manager) != [] and find_goodbye(manager) != []):
        print("Manager said hi and goodbye to client")
    else:
        print("Manager did not said hi and/or goodbye to client")
    print('\n')
    n += 1

Phrases with manager's:
 1.greetings 
 2.name 
 3.company
 4.goodbye


From dialog 1 :
1. ['алло здравствуит']
2. [' меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчиваться']
3. [' меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчиваться']
4. [' всего хорошего до свидание']
Manager said hi and goodbye to client


From dialog 2 :
1. ['алло здравствуит']
2. [' меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчиваться']
3. [' меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчиваться']
4. [' до свидание']
Manager said hi and goodbye to client


From dialog 3 :
1. ['алло здравствуит']
2. [' меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотруднича